In [46]:
from __future__ import division
import psycopg2
import numpy as np
import json
import re
from gensim import corpora, models, similarities
import nltk

In [3]:
conn = psycopg2.connect("host=localhost port=5432 dbname=qa")
cur = conn.cursor()

<h1> Load in text data from SQL </h1>

    catagory text,\
    questionType text,\
    answerType text,\
    asin text,\
    answerTime TimeStamp,\
    unixTime int,\
    question text,\
    answer text\

In [4]:
cur.execute("SELECT question from qa;")
Qresults=cur.fetchall()
cur.execute("SELECT answer from qa;")
Aresults=cur.fetchall()

In [95]:
#def process_line(sentence):
#    filter_text=' '.join(re.findall("[a-z'?]+", sentence.lower()))
#    #return nltk.word_tokenize(filter_text)
#    return filter_text.replace('?',' ? ').split()

def process_line(sentence):
    #step 1 split if we need to
    sentences=re.split(r'[;:!?.-]\s*', sentence)
    result= [re.findall("[a-z']+", sent.lower()) for sent in sentences if \
           re.findall("[a-z']+", sent.lower())!=[]]
    if result==[]:
        result=['']
    return result
stoplist = set('for a of the and to in rt'.split())

<h1> Clean up sentence </h1>

In [ ]:
qs= [[word for word in process_line(sentence[0])[0] if word not in stoplist] for sentence in Qresults]
#want to train model on all questions. Here we only return the first sentnece. To be labeled later

In [101]:
qs[1]

['how', 'well', 'does', 'this', 'work', 'podcasting']

<h1> Clean up the sentences (for twitter) </h1>

In [10]:
Qsentence= [[word for word in sentence[0].lower().split() if word not in stoplist \
                  and re.findall('http',word)==[] and re.findall('@',word)==[]\
                 and re.findall('#',word)==[]] for sentence in Qrestuls]
Asentence= [[word for word in sentence[0].lower().split() if word not in stoplist \
                  and re.findall('http',word)==[] and re.findall('@',word)==[]\
                 and re.findall('#',word)==[]] for sentence in Arestuls]

In [ ]:
Asentence[0][1]

<h1> Train Model </h1>

In [10]:
Qmodel = models.Word2Vec(question_sentence, size=100, window=5, min_count=10, workers=4)

In [12]:
Qmodel.most_similar('can')

[('could', 0.9090049266815186),
 ('must', 0.6884556412696838),
 ('able', 0.6011472940444946),
 ('should', 0.5577580332756042),
 ('cant', 0.521159291267395),
 ('will', 0.5159804224967957),
 ('would', 0.5070073008537292),
 ('intend', 0.5025068521499634),
 ('may', 0.5004855394363403),
 ("'d", 0.4943317472934723)]

In [ ]:
YN_flags=['can I']

<h1> Train for Bigrams </h1>

In [98]:
Qbigram = models.Phrases(qs,min_count=2)
Qbigrams=list(Qbigram[qs])

In [100]:
Qbigrams[:10]

[[u'replacement', u'part', u'delicate', u'piece', u'on', u'zoom_h'],
 [u'how', u'well', u'does', u'this', u'work', u'podcasting'],
 [u'recording', u'phone', u'interview'],
 [u'zoom_h', u'display', u'failure'],
 [u'h', u'price', u'cuts'],
 [u'can', u'this', u'be_mounted', u'wall'],
 [u'i',
  u'was_wondering',
  u'if',
  u'this',
  u'microphone',
  u'would',
  u'work',
  u'with',
  u'cannon',
  u'hd',
  u'vixia_hf',
  u'r',
  u'video',
  u'camera'],
 [u'is', u'this', u'an', u'xlr', u'plug'],
 [u'what', u'is', u'low'],
 [u'has_anyone',
  u'cut',
  u'these',
  u'open',
  u'looked_at',
  u'shielding',
  u'build_quality']]

In [102]:
QmodelB = models.Word2Vec(Qbigrams, size=100, window=5, min_count=10, workers=4)
#AmodelB.most_similar(positive=['finna', 'gonna'], negative=['bro'],topn=20)
#QmodelB.most_similar('hey') #positive=['finna', 'gonna'], negative=['bro'],topn=20)

In [133]:
QmodelB.most_similar("didn'",topn=15)

KeyError: "word 'didn'' not in vocabulary"

In [111]:
#QmodelB['hello']

<h1> Setup logistic regression learning. User first word of sentence to identify type</h1>

<h3> get "labeled" data </h1>

In [128]:
cur.execute("SELECT question from qa WHERE questiontype  = 'yes/no';")
Q_yn=cur.fetchall()
cur.execute("SELECT question from qa WHERE questiontype  = 'open-ended';")
Q_oe=cur.fetchall()

#only evaluate first sentence
qs_yn= [[word for word in process_line(sentence[0])[0] if word not in stoplist] for sentence in Q_yn]
qs_yn_sample=qs_yn[:int(len(qs_yn)/2)]

qs_oe= [[word for word in process_line(sentence[0])[0] if word not in stoplist] for sentence in Q_oe]
qs_oe_sample=qs_oe[:int(len(qs_oe)/2)]

In [130]:
qs_yn_sample[1][0]

'is'

In [1]:
qs_yn_sample[i-1][0]

NameError: name 'qs_yn_sample' is not defined

<h3> setup vars </h3>

In [177]:
all_yn_arrays = np.zeros((len(qs_yn_sample), 100))
all_oe_arrays = np.zeros((len(qs_oe_sample), 100))
all_yn_labels = np.zeros(len(qs_yn_sample))
all_oe_labels = np.zeros(len(qs_oe_sample))

           
#setup training for Y/N questions ... only tranin on the first word!
for i in range(len(qs_yn_sample)):
    if(qs_yn_sample[i]==[]):
        qs_yn_sample[i]=['']
    try:
        all_yn_arrays[i] = QmodelB[qs_yn_sample[i][0]]
    except KeyError:
        all_yn_arrays[i] = all_yn_arrays[i-1]
    all_yn_labels[i] = 1

#setup training for open-ended questions
for ii in range(len(qs_oe_sample)):
    if(qs_oe_sample[ii]==[]):
        qs_oe_sample[ii]=['']
    try:
        all_oe_arrays[ii] = QmodelB[qs_oe_sample[ii][0]]
    except KeyError:
        all_oe_arrays[ii]=all_oe_arrays[ii-1]
    all_oe_labels[ii] = 0

In [178]:
Nyn=len(all_yn_arrays)
Noe=len(all_oe_arrays)

train_arrays = np.vstack((all_yn_arrays[:Nyn/2],all_oe_arrays[:Noe/2]))
train_labels = np.hstack((all_yn_labels[:Nyn/2],all_oe_labels[:Noe/2]))

test_arrays = np.vstack((all_yn_arrays[Nyn/2:],all_oe_arrays[Noe/2:]))
test_labels = np.hstack((all_yn_labels[Nyn/2:],all_oe_labels[Noe/2:]))

In [179]:
# classifier
from sklearn.linear_model import LogisticRegression, LinearRegression

In [180]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [181]:
classifier.score(test_arrays, test_labels)

0.8011458130225837

In [182]:
test_labels

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

<h3> test on real questions </h3>

In [183]:
import warnings
warnings.filterwarnings('ignore')
prediction=[]
for line_arr in qs_yn[-1000:]:
    try:
        prediction.append(classifier.predict(QmodelB[line_arr[0]]))
    except KeyError:
        prediction.append(0)

In [184]:
sum(prediction)/len(prediction)

array([ 0.837])

In [188]:
classifier.predict(QmodelB['can'])

array([ 1.])

In [191]:
prediction=[]
for line_arr in qs_oe[-1000:]:
    try:
        prediction.append(classifier.predict(QmodelB[line_arr[0]]))
    except KeyError:
        prediction.append(0)

In [192]:
sum([item ==0 for item in prediction])/len(prediction)

array([ 0.746])

In [200]:
bag_of_words_yn='is,will,wil,may,might,does,dose,doe,dos,do,can,could,must,shuold,are,would,do,did'.split(',')

In [197]:
bag_of_words_yn

['is',
 'will',
 ' wil',
 'may',
 ' might',
 ' does',
 ' dose',
 ' doe',
 ' dos',
 ' do',
 ' can',
 ' could',
 ' must',
 ' shuold',
 ' are',
 ' would',
 ' do',
 ' did']

In [199]:
'could is a senctence'.split()[0] in bag_of_words_yn

False

<h1> For cluter plot </h1>

In [76]:
########## emoji visual data ##########################################
def clean_result(model_result):
    return [item[0] for item in QmodelB.most_similar('you')],[item[1] for item in QmodelB.most_similar('you')]

def visual(word):
    word=word.lower()
    results,counts=clean_result(QmodelB.most_similar(word))
    source_target=[]
    for result_word in results:
        #append source target, and search next layer
        source_target.append((word,result_word))
        results2,counts2=clean_result(QmodelB.most_similar(result_word))
        for result_word2 in results2:
            source_target.append((result_word,result_word2))
    return [{"source":src,"target":tar,"type":"blah"} for src,tar in source_target]

def tree(word):
    word=word.lower()
    results,counts=clean_result(QmodelB.most_similar(word))
    target=[]
    child_list=[]
    for result_word in results:
        target_child=[]
        target.append(result_word)
        results2,counts2=clean_result(QmodelB.most_similar(result_word))
        for result_word2 in results2:
            target_child.append(result_word2)
        child_list.append(target_child)
    return target,child_list
    #return [{"source":src,"size":tar} for src,tar in source_target]

In [81]:
word='you'
target,child_list=tree(word)

In [85]:
{"name":word,"children":[{"name":tar,"children":[{"name":child,"group":3} for child in child_l] }\
                           for tar,child_l in zip(target,child_list)]}

{'children': [{'children': [{'group': 3, 'name': u'u'},
    {'group': 3, 'name': u'we'},
    {'group': 3, 'name': u'i'},
    {'group': 3, 'name': u'yall'},
    {'group': 3, 'name': u'anyone'},
    {'group': 3, 'name': u'anybody'},
    {'group': 3, 'name': u'ability'},
    {'group': 3, 'name': u"y'all"},
    {'group': 3, 'name': u'your'},
    {'group': 3, 'name': u'yourself'}],
   'name': u'u'},
  {'children': [{'group': 3, 'name': u'u'},
    {'group': 3, 'name': u'we'},
    {'group': 3, 'name': u'i'},
    {'group': 3, 'name': u'yall'},
    {'group': 3, 'name': u'anyone'},
    {'group': 3, 'name': u'anybody'},
    {'group': 3, 'name': u'ability'},
    {'group': 3, 'name': u"y'all"},
    {'group': 3, 'name': u'your'},
    {'group': 3, 'name': u'yourself'}],
   'name': u'we'},
  {'children': [{'group': 3, 'name': u'u'},
    {'group': 3, 'name': u'we'},
    {'group': 3, 'name': u'i'},
    {'group': 3, 'name': u'yall'},
    {'group': 3, 'name': u'anyone'},
    {'group': 3, 'name': u'anybody

<h1> load and save </h1>

In [105]:
QmodelB.save('QmodelB_update')

In [47]:
QmodelB=models.Word2Vec.load('QmodelB')